In [1]:
# !echo $RAYCLUSTER_AUTOSCALER_HEAD_SVC_PORT_10001_TCP_ADDR
# !echo $RAYCLUSTER_AUTOSCALER_HEAD_SVC_PORT_10001_TCP_PORT

In [1]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [2]:
import ray
import time

In [3]:
# from utils import logger

In [9]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from utils import helper, config, rayer
from ml.models.base.brisk_xgboost import BriskXGBoost
from ml.models.base.slug_xgboost import SlugXGBoost
from ml.models.base.slug_ann import SlugANN
from ml.models.base.slug_rf import SlugRF
from ml.models.base.slug_knn import SlugKNN

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common
from sklearn.preprocessing import StandardScaler


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
rayer.get_global_cluster(num_cpus=4)

In [6]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-18 02:37:31.910231 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 2 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/5.0 CPU
 0.00/16.764 GiB memory
 0.00/4.898 GiB object_store_memory

Demands:
 {}: 1+ pending tasks/actors
 {'CPU': 1}: 4+ from request_resources()


In [7]:
# ray.autoscaler.sdk.request_resources(num_cpus=0)

In [10]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)

        
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

ss = StandardScaler()
X_train_scalar = pd.DataFrame(ss.fit_transform(X_train), columns = X_train.columns)
X_test_scalar = pd.DataFrame(ss.fit_transform(X_test), columns = X_test.columns)



In [12]:

slug_ann = SlugANN('slug_ann', X_train_scalar, X_test_scalar, y_train, y_test)
slug_ann.epochs = 10
slug_ann.n_trials = 20 


In [13]:
slug_ann.fetch_model()

2022-11-18T02:39:47CST : INFO : slug_ann : __discover_model__ : 225 : Message : brisk_xgb: Starting train for trials:20 with epochs: 10
2022-11-18T02:39:55CST : INFO : slug_ann : __discover_model__ : 238 : Message : brisk_xgb: Number of trials: 20
2022-11-18T02:39:55CST : INFO : slug_ann : __discover_model__ : 240 : Message : Best trial: 19
2022-11-18T02:39:55CST : INFO : slug_ann : __discover_model__ : 247 : Message : Sequential(
  (0): Linear(in_features=40, out_features=195, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.32617147222054993, inplace=False)
  (3): Linear(in_features=195, out_features=256, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.32617147222054993, inplace=False)
  (6): Linear(in_features=256, out_features=199, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.32617147222054993, inplace=False)
  (9): Linear(in_features=199, out_features=188, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.32617147222054993, inplace=False)
  (12): Linear(in_features=188, out_features=1, bias=Tru

Sequential(
  (0): Linear(in_features=40, out_features=195, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.32617147222054993, inplace=False)
  (3): Linear(in_features=195, out_features=256, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.32617147222054993, inplace=False)
  (6): Linear(in_features=256, out_features=199, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.32617147222054993, inplace=False)
  (9): Linear(in_features=199, out_features=188, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.32617147222054993, inplace=False)
  (12): Linear(in_features=188, out_features=1, bias=True)
  (13): ReLU()
)

In [14]:
slug_ann.score

[0.6290799664612925, 0.5690131407573895]

(scheduler +6m3s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +6m3s) Adding 1 node(s) of type small-group.
(scheduler +6m9s) Resized to 7 CPUs.
(scheduler +7m15s) Removing 1 nodes of type small-group (idle).
(scheduler +7m26s) Resized to 5 CPUs.


Log channel is reconnecting. Logs produced while the connection was down can be found on the head node of the cluster in `ray_client_server_[port].out`
2022-11-18 02:58:39,923	WARNING dataclient.py:396 -- Encountered connection issues in the data channel. Attempting to reconnect.
2022-11-18 02:58:49,024	ERROR dataclient.py:323 -- Unrecoverable error in data channel.
Unexpected exception:
Traceback (most recent call last):
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/ray/util/client/logsclient.py", line 67, in _log_main
    for record in log_stream:
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-275852-64-wasif-dev-ray/lib/python3.9/site-packages/grpc/_channel.py", line 426, in __next__
    return self._next()
  File "/home/conda/admin/807c952d1ebb2e498414dc4b80a90b96c80ef2ef433f07c7a7381b5e38e2b5fa-20221110-224801-

In [16]:

slug_xgb = SlugXGBoost('slug_xgb', X_train_scalar, X_test_scalar, y_train, y_test)
slug_xgb.boosted_round = 50
slug_xgb.n_trials = 1000


In [17]:
slug_xgb.fetch_model()

2022-11-17T13:52:14CST : INFO : slug_xgboost : __discover_model__ : 168 : Message : slug_xgb: Starting training for trials:1000, boosted rounds: 50, max depth: 10
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 181 : Message : slug_xgb: Number of trials: 1000
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 183 : Message : Best trial:611
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 185 : Message :   Params: 
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     lambda 0.019399866419368653
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     alpha 0.0020906308336665715
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     eta 0.3727957591519319
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     gamma 0.0033926747144862554
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ :

In [36]:

slug_rf = SlugRF('slug_rf', X_train_scalar, X_test_scalar, y_train, y_test)
slug_rf.max_n_estimators = 5000
slug_rf.n_trials = 100
slug_rf.max_depth = 200

In [37]:
best_fit = slug_rf.fetch_model()

2022-11-17T14:56:36CST : INFO : slug_rf : __discover_model__ : 105 : Message : slug_rf: Starting training for trials:100, max depth: 200, max estimators 5000
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 122 : Message : slug_rf: Number of trials: 100
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 124 : Message : Best trial:74
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 126 : Message :   Params: 
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     n_estimators 62
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     max_depth 26
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     min_samples_split 52
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     min_samples_leaf 34
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 142 : Message :   test r2 score: 0.3992977208659255
2022-11-17T14:59:59CST : INFO : 

In [35]:
slug_rf.score

[0.5264056467634264, 0.39565984792823017]